In [2]:
import numpy as np
import cv2

In [2]:
#corners = [[99, 67], [1933, 111], [1, 1359], [2048, 1351]]
corners = [[1163, 529], [3573, 891], [193, 1755], [3549, 2705]]

def getBoard(img):
    width, height = 800, 600
    oldPoints = np.float32(corners)
    newPoints = np.float32([[0,0], [width,0], [0,height], [width,height]])
    m = cv2.getPerspectiveTransform(oldPoints, newPoints)
    out = cv2.warpPerspective(img, m, (width, height))
    
    return out

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = cv2.imread('dataset/original/3.jpg')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

cv2.imshow('image',thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2

image = cv2.imread('dataset/original/1.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
canny = cv2.Canny(gray, 120, 255, 1)

corners = cv2.goodFeaturesToTrack(canny,4,0.5,50)
listx = []
listy = []
for corner in corners:
    x,y = corner.ravel()
    listx.append(x)
    listy.append(y)
    cv2.circle(image,(int(x),int(y)),10,(36,255,12),10)

print(listx)
print(listy)
#cv2.imshow('canny', canny)
cv2.imshow('image', image)
cv2.waitKey()

[3066.0, 2559.0, 386.0, 2108.0]
[603.0, 2427.0, 1400.0, 1452.0]


In [ ]:
#Tworzenie masek z zaznaczonymi rogami
for i in range(60):
    name = str(i+1)+'_done'
    img = cv2.imread('dataset/points/'+name+'.jpg')
    for j in range(img.shape[0]):
        for k in range(img.shape[1]):
            if not img[j][k][0] == img[j][k][1] == img[j][k][2] == 255:
                img[j][k][0] = img[j][k][1] = img[j][k][2] = 0
    cv2.imwrite('dataset/mask/'+name+'_mask.jpg', img)
                

In [ ]:
import math
import numpy as np
import cv2
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

for i in range(13,60):
    name = str(i+1)+'_done_mask'
    img = cv2.imread('dataset/mask/'+name+'.jpg')
    corners = []
    for j in range(img.shape[0]):
        for k in range(img.shape[1]):
            if img[j][k][0] == img[j][k][1] == img[j][k][2] == 255:
                if len(corners) == 0:
                    corners.append([j,k])
                else:
                    for ind in range(len(corners)):
                        if math.dist([j,k], corners[ind]) < 50:
                            break
                        if ind == len(corners)-1:
                            corners.append([j,k])
    temp = corners[3]
    corners[3] = corners[2]
    corners[2] = temp
    polygon = Polygon(corners)
    newImg =  np.zeros((img.shape[0], img.shape[1]))
    for j in range(newImg.shape[0]):
        for k in range(newImg.shape[1]):
            if polygon.contains(Point(j,k)):
                newImg[j][k] = 255
                
    cv2.imwrite('dataset/new_mask/'+name+'_mask.jpg', newImg)



In [1]:
import numpy as np
import cv2
from keras.models import Model, load_model
import tensorflow as tf

img = cv2.imread('dataset/original/46.jpg')
img = cv2.resize(img, (512,512))


model = load_model('C:/Users/mmalk/Ticket-to-ride/dataset/model2.h5')
newframe = np.expand_dims(img, axis=0)
imgNew = model.predict(newframe, verbose=0)
print(imgNew.shape)
cv2.imshow('i',imgNew[0])
cv2.waitKey(0)
#cv2.destroyAllWindows()

(1, 512, 512, 1)


16

In [11]:
config = model.get_config() # Returns pretty much every information about your model
print(config["layers"][0]["config"]["batch_input_shape"])

(None, 512, 512, 3)
